In [4]:
from linkedin_scraper import Company, actions, Person
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [5]:
# driver = webdriver.Chrome()
email = "santlalc27@gmail.com"
password = "Sant@123"

### Service account for

##### https://docs.google.com/spreadsheets/d/1vkbxhky9ek5JDZhyRpjD3Em8pxp0q9rSUh6Mfa5UKIA/edit?gid=0#gid=0

In [15]:
import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = "/home/abhay/Desktop/Personal/Learning/Credentials/ordinal-quarter-387322-7194228669a8.json"

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

SHEET_NAME = "Linkedin"

#

sheet = client.open(SHEET_NAME).sheet1  # opens first sheet

data = sheet.get_all_records()

print("✅ Successfully connected! Sample data:")
for row in data[:]:
    print(row)


✅ Successfully connected! Sample data:
{'subject': 'Full stack developer', 'email': 'abhaysingh@iitbhilai.ac.in', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Abhay'}
{'subject': 'Full stack developer', 'email': 'abhaysinghpatel690@gmail.com', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Singh'}
{'subject': 'Full stack developer', 'email': 'anshukumar93.40@gmail.com', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Anshu'}
{'subject': 'Full stack developer', 'email': 'gauranshurathee3@gmail.com', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Gauranshu'}
{'subject': 'Full stack developer', 'email': 'sahanmondal2002@gmail.com', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Gauranshu'}
{'subject': 'Full stack developer', 'email': 'pjaishree8@gmail.com', 'template_name': 'resume', 'attachment_no': 'Abhay', 'name': 'Sachi'}
{'subject': 'Full stack developer', 'email': 'pravinshankhapal7@gmail.com', 'template_nam

In [16]:
import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = "/home/abhay/Desktop/Personal/Learning/Credentials/ordinal-quarter-387322-7194228669a8.json"

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

SHEET_NAME = "Linkedin"

# Cache spreadsheet object here
SPREADSHEET = client.open(SHEET_NAME)

SHEET_MAPPING = {
    1: "HR",
    2: "accounts",
    3: "profile",
    4: "jobs",
}

def get_sheet_by_number(sheet_number):
    """Return gspread worksheet based on mapped sheet number."""
    sheet_name = SHEET_MAPPING.get(sheet_number)

    if not sheet_name:
        raise ValueError(f"No sheet mapping found for number: {sheet_number}")

    # Use cached spreadsheet instead of opening again
    return SPREADSHEET.worksheet(sheet_name)


In [ ]:
from pydantic import BaseModel, EmailStr, HttpUrl, field_validator
from typing import Optional, List
from datetime import datetime

class JobModel(BaseModel):
    title: str
    company: str
    location: Optional[str] = None
    job_type: Optional[str] = None
    url: HttpUrl
    
    @field_validator('url')
    @classmethod
    def validate_linkedin_url(cls, v):
        url_str = str(v)
        if not url_str.startswith('https://www.linkedin.com/'):
            raise ValueError('URL must be a valid LinkedIn URL')
        return v
    
    @field_validator('title', 'company')
    @classmethod
    def validate_not_na(cls, v):
        if v == "N/A" or not v.strip():
            raise ValueError('Title and company cannot be N/A or empty')
        return v.strip()

# Simple helper functions
# def get_existing_urls(sheet) -> set:
#     """Get existing URLs"""
#     try:
#         data = sheet.get_all_records()
#         return {row.get('URL') for row in data if row.get('URL')}
#     except:
#         return set()

def validate_jobs(jobs_data, existing_urls):
    """Simple validation"""
    validated = []
    
    for job in jobs_data:
        try:
            if job.get('url') in existing_urls:
                continue
                
            validated_job = JobModel(
                title=job.get('title', 'N/A'),
                company=job.get('company', 'N/A'),
                location=job.get('location') if job.get('location') != 'N/A' else None,
                job_type=job.get('job_type') if job.get('job_type') != 'N/A' else None,
                url=job.get('url', '')
            )
            validated.append(validated_job)
            
        except Exception as e:
            print(f"❌ Invalid job: {e}")
            continue
    
    return validated

def upload_to_sheet(sheet, jobs):
    """Upload jobs to sheet"""
    if not jobs:
        return False
        
    try:
        rows = []
        for job in jobs:
            row = [
                job.title,
                job.company,
                job.location or "",
                job.job_type or "",
                str(job.url),
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ]
            rows.append(row)
        
        sheet.append_rows(rows)
        print(f"✅ Uploaded {len(jobs)} jobs!")
        return True
    except Exception as e:
        print(f"❌ Upload failed: {e}")
        return False

In [17]:
sheet = get_sheet_by_number(4)
columns = sheet.row_values(1)
data = sheet.get_all_records()
print(f"✅ Accessed sheet: {sheet.title} , {sheet}")
print("columns:", columns)
for row in data:
    print(row)

✅ Accessed sheet: jobs , <Worksheet 'jobs' id:756071209>
columns: ['Title', 'Company', 'Location', 'Job Type', 'URL', 'created_at']
{'Title': 'Frontend Developer\nFrontend Developer', 'Company': 'GoodSpace AI', 'Location': '', 'Job Type': 'software developer', 'URL': 'https://www.linkedin.com/jobs/view/4341822354/?eBP=CwEAAAGapnNvJlTsDxNFNBfAqfQ1aT_vr7sFLX0CLtvTdorBKr1eE5x2rFlpu_52exA5rxL8XDkSz1HmMRDQrwjy0n_BjfKIhar72INrfT7ogmDfvkul3kWM6nyTpfhTM3hr6EjhYo4-c6WD91he0LKbB4v-KLp0mLj8K-_fPG86JgVA2iZInza9SXifguZy2rK-JiGA91CsYpyl6B0VjebEvnHBQmJ-HljMIVrQ1-5cVy5LoKOHGZt6m-yo0GzaFMeARKA_Jf0Opb3tZ2sr3ZvQUvZDgtkEWEoCz_JV6Bg98wvVseJDsI2VkQSEfT_Bn20-gQuAEhgoZexNrFoLiRY-fQV9xeIULfAPHlLXMVOSPBvi23CHt1zpU-6VsYKB44drK0LHpRviexjwm2SVX82Tk8yndX4mTSmiVvlxkBKPpLXOb52UzkRU7fIYAz0mswGTtlVqWq1SdrT4P8qcIFfYZO02cuVkjVMGrLTr_EQg7Z3NeDCwYpcjY4iGI9FJSbH_K_WNPYprCdKQ36-5WmQXw-m3&refId=%2Ba3OdWAamUk8%2FcgzimLzbA%3D%3D&trackingId=pI9BMVsGsThBEudXkPu6AQ%3D%3D&trk=flagship3_search_srp_jobs', 'created_at': '2025-11-21 19

In [8]:
job_keywords = [
        "software developer",
        "web developer", 
        "python developer",
        "frontend developer",
        "backend developer",
        "full stack developer",
        "data scientist",
        "machine learning engineer"
    ]
BASE_URL = "https://www.linkedin.com/jobs/search/"
page_number = 5
def get_jobs():
    driver = webdriver.Chrome()
    time.sleep(5)
    try:
        actions.login(driver, email, password)
    except TimeoutException:
        time.sleep(30)

    print("Logged in successfully")
    all_jobs_data = []
    try:
        total_jobs_found = 0
        seen_job_urls = set() 
        for idx, job_keyword in enumerate(job_keywords, 1):
            keyword_jobs_count = 0
            for page in range(page_number):
                start_param = page * 25
                encoded_keyword = job_keyword.replace(" ", "%20")
                search_url = f"{BASE_URL}?f_TPR=r86400&keywords={encoded_keyword}&start={start_param}"
                driver.get(search_url)
                time.sleep(5)
                last_height = driver.execute_script("return document.body.scrollHeight")
                scroll_attempts = 0
                max_scroll_attempts = 2 
                while scroll_attempts < max_scroll_attempts:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        scroll_attempts += 1
                    else:
                        scroll_attempts = 0  
                    last_height = new_height
                try:
                    job_cards = WebDriverWait(driver, 15).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-job-id]"))
                    )
                    
                    page_jobs = []
                    
                    for i, job_card in enumerate(job_cards, 1):
                        try:
                            job_title = "N/A"
                            job_url = "N/A"
                            title_selectors = [
                                "h3 a span[title]",
                                "h3 a",
                                ".base-search-card__title a",
                                ".job-card-container__link",
                                "a[data-tracking-control-name*='job']",
                                ".base-card__full-link",
                                "a span[aria-hidden='true']"
                            ]
                            
                            for selector in title_selectors:
                                try:
                                    job_title_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                    if job_title_elem:
                                        job_title = job_title_elem.text.strip() or job_title_elem.get_attribute("title") or "N/A"
                                        if job_title_elem.tag_name == "a":
                                            job_url = job_title_elem.get_attribute("href") or "N/A"
                                        else:
                                            # Find parent link
                                            parent_link = job_title_elem.find_element(By.XPATH, "./ancestor::a[1]")
                                            job_url = parent_link.get_attribute("href") if parent_link else "N/A"
                                        if job_title != "N/A":
                                            break
                                except:
                                    continue
                            company_name = "N/A"
                            company_selectors = [
                                # Primary company selectors
                                ".base-search-card__subtitle a",
                                ".job-search-card__subtitle-link",
                                ".job-card-container__company-name",
                                "h4 a",

                                ".base-search-card__subtitle",
                                ".job-search-card__subtitle",
                                "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                                "span[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                                
                                "[data-tracking-control-name*='company']",
                                ".job-card-container__primary-description",
                                "a[href*='/company/']",
                                
                                ".artdeco-entity-lockup__subtitle",
                                ".job-result-card__subtitle",
                                ".jobs-search-results-list__subtitle"
                            ]
                            
                            for selector in company_selectors:
                                try:
                                    company_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                    if company_elem:
                                        # Try text first, then aria-label, then title attribute
                                        company_text = (company_elem.text.strip() or 
                                                    company_elem.get_attribute("aria-label") or 
                                                    company_elem.get_attribute("title") or "").strip()
                                        
                                        # Clean up common prefixes/suffixes
                                        if company_text and company_text not in ["N/A", "", "Company"]:
                                            # Remove common prefixes like "Company: "
                                            company_text = company_text.replace("Company:", "").strip()
                                            company_name = company_text
                                            break
                                except:
                                    continue
                            
                            if company_name == "N/A":
                                try:
                                    # Look for company info in sibling or parent elements
                                    all_text_elements = job_card.find_elements(By.CSS_SELECTOR, "a, span, div")
                                    for elem in all_text_elements[:10]:  # Check first 10 elements only
                                        elem_text = elem.text.strip()
                                        href = elem.get_attribute("href") or ""
                                        
                                        # If element links to a company page, it's likely the company name
                                        if "/company/" in href and elem_text and len(elem_text) < 100:
                                            company_name = elem_text
                                            break
                                except:
                                    pass
                            
                            # Try to find location with multiple selectors
                            location = "N/A"
                            location_selectors = [
                                ".job-search-card__location",
                                ".base-search-card__metadata .job-card-container__metadata-item",
                                ".base-search-card__metadata",
                                "[data-tracking-control-name*='location']"
                            ]
                            
                            for selector in location_selectors:
                                try:
                                    location_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                    if location_elem:
                                        location = location_elem.text.strip()
                                        if location:
                                            break
                                except:
                                    continue
                            
                            # Check for duplicates using URL
                            if job_url != "N/A" and job_url not in seen_job_urls:
                                seen_job_urls.add(job_url)
                                
                                # Store job data
                                job_data = {
                                    'title': job_title,
                                    'company': company_name,
                                    'location': location,
                                    'url': job_url,
                                    'job_type': job_keyword,
                                    'page': page + 1
                                }
                                page_jobs.append(job_data)
                            
                        except Exception as job_error:
                            continue  
                    
                    all_jobs_data.extend(page_jobs)
                    keyword_jobs_count += len(page_jobs)
                except Exception as page_error:
                    print(f"    ❌ Error on page {page + 1}: {str(page_error)[:50]}...")
                    continue
    except Exception as e:
        print(f"❌ Error during scraping: {e}")
        driver.quit()
    return all_jobs_data    

In [9]:
jobs = get_jobs()

Logged in successfully


In [10]:
print(len(jobs))
for job in jobs:
    # print(job.get("company"))
    print(job,"end=")

280
{'title': 'Software Developer (Fresher)\nSoftware Developer (Fresher)', 'company': 'Tandemloop Technologies', 'location': 'N/A', 'url': 'https://www.linkedin.com/jobs/view/4323627787/?eBP=BUDGET_EXHAUSTED_JOB&refId=MoHu985jEQXGYDxHFaVNSw%3D%3D&trackingId=7W5W1kuKZIqFTh9L0qE%2BQg%3D%3D&trk=flagship3_search_srp_jobs', 'job_type': 'software developer', 'page': 1} end=
{'title': 'Frontend Developer\nFrontend Developer with verification', 'company': 'Swiggy', 'location': 'N/A', 'url': 'https://www.linkedin.com/jobs/view/4337624408/?eBP=BUDGET_EXHAUSTED_JOB&refId=MoHu985jEQXGYDxHFaVNSw%3D%3D&trackingId=nguSTfPydJ5hpg9c9Kqlgg%3D%3D&trk=flagship3_search_srp_jobs', 'job_type': 'software developer', 'page': 1} end=
{'title': 'Software Engineer Intern\nSoftware Engineer Intern with verification', 'company': 'Red Hat', 'location': 'N/A', 'url': 'https://www.linkedin.com/jobs/view/4323578663/?eBP=BUDGET_EXHAUSTED_JOB&refId=MoHu985jEQXGYDxHFaVNSw%3D%3D&trackingId=vRajKnU%2FzRQYr2y4riG97g%3D%3D&t

In [11]:
from pydantic import BaseModel, HttpUrl, field_validator
from typing import Optional
from datetime import datetime

# -----------------------------
#   MODEL
# -----------------------------
class JobModel(BaseModel):
    title: str
    company: str
    location: Optional[str] = None
    job_type: Optional[str] = None
    url: HttpUrl
    
    @field_validator('url')
    @classmethod
    def validate_linkedin_url(cls, v):
        if not str(v).startswith('https://www.linkedin.com/'):
            raise ValueError('URL must be a valid LinkedIn URL')
        return v
    
    @field_validator('title', 'company')
    @classmethod
    def validate_not_na(cls, v):
        if v == "N/A" or not v.strip():
            raise ValueError('Title and company cannot be empty or N/A')
        return v.strip()


# -----------------------------
#   GET EXISTING URLs
# -----------------------------
def get_existing_urls(sheet) -> set:
    data = sheet.get_all_records()
    return {row.get('URL') for row in data if row.get('URL')}


# -----------------------------
#   VALIDATE JOBS
# -----------------------------
def validate_jobs(jobs_data, existing_urls):
    validated = []
    
    for job in jobs_data:
        try:
            url = job.get('url')
            if url in existing_urls:
                print(f"⚠️ Skipped duplicate: {url}")
                continue
                
            model = JobModel(
                title=job.get('title'),
                company=job.get('company'),
                location=None if job.get('location') == "N/A" else job.get('location'),
                job_type=None if job.get('job_type') == "N/A" else job.get('job_type'),
                url=url
            )
            validated.append(model)
            
        except Exception as e:
            print(f"❌ Invalid job skipped: {e}")
            continue
    
    return validated


# -----------------------------
#   UPLOAD TO SHEET
# -----------------------------
def upload_to_sheet(sheet, jobs):
    if not jobs:
        print("⚠️ No new jobs to upload.")
        return False

    try:
        rows = []

        for job in jobs:
            row = [
                job.title,
                job.company,
                job.location or "",
                job.job_type or "",
                str(job.url),
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ]
            rows.append(row)
        
        sheet.append_rows(rows)
        print(f"✅ Uploaded {len(jobs)} jobs!")
        return True

    except Exception as e:
        print(f"❌ Upload failed: {e}")
        return False
def clear_sheet_except_header(sheet):
    """
    Deletes all rows except the first row (header).
    """
    try:
        all_values = sheet.get_all_values()
        total_rows = len(all_values)
        print(f"Total rows before cleaning: {total_rows}")

        if total_rows <= 1:
            print("🧼 Sheet already clean (only header exists).")
            return True

        # Delete rows from 2 to last
        sheet.delete_rows(2, total_rows - 1)
        print(f"🧹 Cleared sheet: removed {total_rows - 1} rows, header kept.")

        return True

    except Exception as e:
        print(f"❌ Error during sheet cleaning: {e}")
        return False


In [18]:
def main(jobs_data):
    print("📄 Starting job upload pipeline...")

    # 1️⃣ Load sheet
    sheet = get_sheet_by_number(4)
    print(f"✅ Accessed sheet: {sheet.title}")

    clear_sheet_except_header(sheet)

    # 2️⃣ Get existing URLs
    existing_urls = get_existing_urls(sheet)
    print(f"🔎 Existing jobs: {len(existing_urls)} URLs found")

    # 3️⃣ Validate + filter
    valid_jobs = validate_jobs(jobs_data, existing_urls)
    print(f"🧹 Valid jobs after filtering: {len(valid_jobs)}")

    # 4️⃣ Upload
    if valid_jobs:
        upload_to_sheet(sheet, valid_jobs)
    else:
        print("⚠️ No valid new jobs to upload")

    print("🎉 Done.")


In [19]:
main(jobs)

📄 Starting job upload pipeline...
✅ Accessed sheet: jobs
Total rows before cleaning: 8
🧹 Cleared sheet: removed 7 rows, header kept.
🔎 Existing jobs: 1 URLs found
🧹 Valid jobs after filtering: 280
✅ Uploaded 280 jobs!
🎉 Done.


In [ ]:
def get_existing_urls(sheet) -> set:
    """Return existing URLs from sheet"""
    data = sheet.get_all_records()
    return {row.get('URL') for row in data if row.get('URL')}

In [ ]:

driver = webdriver.Chrome()

# Wait for page to load before attempting login
time.sleep(5)  

# email = "santlalc27@gmail.com"
# password = "Sant@123"

try:
    actions.login(driver, email, password)
except TimeoutException:
    time.sleep(30)  # 30 seconds to handle verification

print("Proceeding with job search scraping...")

try:
    # Define multiple job keywords to search
    job_keywords = [
        "software developer",
        # "web developer", 
        # "python developer",
        # "frontend developer",
        # "backend developer",
        # "full stack developer",
        # "data scientist",
        # "machine learning engineer"
    ]
    
    all_jobs_data = []
    total_jobs_found = 0
    seen_job_urls = set()  # To avoid duplicates
    
    print(f"🚀 Starting multi-job search for {len(job_keywords)} different job types...")
    
    # Loop through each job keyword
    for job_idx, job_keyword in enumerate(job_keywords, 1):
        print(f"\n🔍 [{job_idx}/{len(job_keywords)}] Searching for: '{job_keyword}'")
        print("=" * 60)
        
        keyword_jobs_count = 0
        
        # Loop through first 5 pages for each job keyword
        for page in range(5):
            start_param = page * 25
            # URL encode the job keyword
            encoded_keyword = job_keyword.replace(" ", "%20")
            job_search_url = f"https://www.linkedin.com/jobs/search-results/?f_TPR=r86400&keywords={encoded_keyword}&start={start_param}"
            
            print(f"  📄 Page {page + 1}/5 - {job_keyword}")
            
            driver.get(job_search_url)
            time.sleep(8)  # Wait for page to load
            
            # Scroll to load more jobs on the page
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_attempts = 0
            max_scroll_attempts = 2  # Reduced for efficiency
            
            while scroll_attempts < max_scroll_attempts:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    scroll_attempts += 1
                else:
                    scroll_attempts = 0  # Reset if we found new content
                last_height = new_height
            
            # Get job cards after scrolling
            try:
                job_cards = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-job-id]"))
                )
                
                page_jobs = []
                
                for i, job_card in enumerate(job_cards, 1):
                    try:
                        # Try multiple possible selectors for job title
                        job_title = "N/A"
                        job_url = "N/A"
                        
                        # Try various selectors for job title and link
                        title_selectors = [
                            "h3 a span[title]",
                            "h3 a",
                            ".base-search-card__title a",
                            ".job-card-container__link",
                            "a[data-tracking-control-name*='job']",
                            ".base-card__full-link",
                            "a span[aria-hidden='true']"
                        ]
                        
                        for selector in title_selectors:
                            try:
                                job_title_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if job_title_elem:
                                    job_title = job_title_elem.text.strip() or job_title_elem.get_attribute("title") or "N/A"
                                    if job_title_elem.tag_name == "a":
                                        job_url = job_title_elem.get_attribute("href") or "N/A"
                                    else:
                                        # Find parent link
                                        parent_link = job_title_elem.find_element(By.XPATH, "./ancestor::a[1]")
                                        job_url = parent_link.get_attribute("href") if parent_link else "N/A"
                                    if job_title != "N/A":
                                        break
                            except:
                                continue
                        
                        # Try to find company name with comprehensive selectors
                        company_name = "N/A"
                        company_selectors = [
                            # Primary company selectors
                            ".base-search-card__subtitle a",
                            ".job-search-card__subtitle-link",
                            ".job-card-container__company-name",
                            "h4 a",
                            
                            # Alternative company selectors
                            ".base-search-card__subtitle",
                            ".job-search-card__subtitle",
                            "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            "span[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            
                            # Generic selectors
                            "[data-tracking-control-name*='company']",
                            ".job-card-container__primary-description",
                            "a[href*='/company/']",
                            
                            # Fallback selectors
                            ".artdeco-entity-lockup__subtitle",
                            ".job-result-card__subtitle",
                            ".jobs-search-results-list__subtitle"
                        ]
                        
                        for selector in company_selectors:
                            try:
                                company_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if company_elem:
                                    # Try text first, then aria-label, then title attribute
                                    company_text = (company_elem.text.strip() or 
                                                  company_elem.get_attribute("aria-label") or 
                                                  company_elem.get_attribute("title") or "").strip()
                                    
                                    # Clean up common prefixes/suffixes
                                    if company_text and company_text not in ["N/A", "", "Company"]:
                                        # Remove common prefixes like "Company: "
                                        company_text = company_text.replace("Company:", "").strip()
                                        company_name = company_text
                                        break
                            except:
                                continue
                        
                        # If still no company found, try finding it in nearby elements
                        if company_name == "N/A":
                            try:
                                # Look for company info in sibling or parent elements
                                all_text_elements = job_card.find_elements(By.CSS_SELECTOR, "a, span, div")
                                for elem in all_text_elements[:10]:  # Check first 10 elements only
                                    elem_text = elem.text.strip()
                                    href = elem.get_attribute("href") or ""
                                    
                                    # If element links to a company page, it's likely the company name
                                    if "/company/" in href and elem_text and len(elem_text) < 100:
                                        company_name = elem_text
                                        break
                            except:
                                pass
                        
                        # Try to find location with multiple selectors
                        location = "N/A"
                        location_selectors = [
                            ".job-search-card__location",
                            ".base-search-card__metadata .job-card-container__metadata-item",
                            ".base-search-card__metadata",
                            "[data-tracking-control-name*='location']"
                        ]
                        
                        for selector in location_selectors:
                            try:
                                location_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if location_elem:
                                    location = location_elem.text.strip()
                                    if location:
                                        break
                            except:
                                continue
                        
                        # Check for duplicates using URL
                        if job_url != "N/A" and job_url not in seen_job_urls:
                            seen_job_urls.add(job_url)
                            
                            # Store job data
                            job_data = {
                                'title': job_title,
                                'company': company_name,
                                'location': location,
                                'url': job_url,
                                'job_type': job_keyword,
                                'page': page + 1
                            }
                            page_jobs.append(job_data)
                        
                    except Exception as job_error:
                        continue  # Skip problematic jobs
                
                all_jobs_data.extend(page_jobs)
                keyword_jobs_count += len(page_jobs)
                
            except Exception as page_error:
                print(f"    ❌ Error on page {page + 1}: {str(page_error)[:50]}...")
                continue
            
            # Small delay between pages
            time.sleep(2)
        
        print(f"  ✅ Found {keyword_jobs_count} unique jobs for '{job_keyword}'")
        total_jobs_found += keyword_jobs_count
        
        # Delay between different job searches
        time.sleep(3)
    
    # Display all collected jobs organized by job type
    print(f"\n🎯 FINAL RESULTS: {total_jobs_found} UNIQUE JOBS FROM {len(job_keywords)} JOB TYPES")
    print("=" * 100)
    
    # Group jobs by job type for better organization
    jobs_by_type = {}
    for job in all_jobs_data:
        job_type = job['job_type']
        if job_type not in jobs_by_type:
            jobs_by_type[job_type] = []
        jobs_by_type[job_type].append(job)
    
    # Display summary by job type
    print(f"\n📊 SUMMARY BY JOB TYPE:")
    for job_type, jobs in jobs_by_type.items():
        print(f"  • {job_type}: {len(jobs)} jobs")
    
    # Display all jobs
    job_counter = 1
    for job_type, jobs in jobs_by_type.items():
        print(f"\n\n🔹 {job_type.upper()} ({len(jobs)} jobs)")
        print("-" * 60)
        
        for job in jobs:
            print(f"{job_counter}. {job['title']}")
            print(f"    Company: {job['company']}")
            print(f"    Location: {job['location']}")
            print(f"    LinkedIn URL: {job['url']}" if len(job['url']) > 80 else f"    LinkedIn URL: {job['url']}")
            print()
            job_counter += 1
    
    print(f"🎉 Successfully scraped {total_jobs_found} unique job listings from {len(job_keywords)} job types!")
    print(f"📊 Jobs were collected from up to 5 pages per job type.")
    print(f"🚫 Duplicates removed: {len(seen_job_urls) - total_jobs_found} duplicate URLs filtered out.")
        
except TimeoutException as te:
    print(f"Timeout error: Could not load job search page")
    print("This might be due to LinkedIn's anti-scraping measures or slow page loading.")
    
except Exception as e:
    print(f"Error during job search: {e}")
    print("LinkedIn may have updated their page structure or implemented new anti-scraping measures.")
    
finally:
    # Always close the driver
    try:
        input("Press Enter to close the browser...")  # Keep browser open to see results
        driver.quit()
        print("Browser closed successfully.")
    except:
        print("Browser was already closed.")

Proceeding with job search scraping...
🚀 Starting multi-job search for 1 different job types...

🔍 [1/1] Searching for: 'software developer'
  📄 Page 1/5 - software developer
  📄 Page 2/5 - software developer
  📄 Page 3/5 - software developer
  📄 Page 4/5 - software developer
  📄 Page 5/5 - software developer
  ✅ Found 35 unique jobs for 'software developer'

🎯 FINAL RESULTS: 35 UNIQUE JOBS FROM 1 JOB TYPES

📊 SUMMARY BY JOB TYPE:
  • software developer: 35 jobs


🔹 SOFTWARE DEVELOPER (35 jobs)
------------------------------------------------------------
1. Software Engineer
    Company: Microsoft
    Location: N/A
    LinkedIn URL: https://www.linkedin.com/jobs/search-results/?currentJobId=4341902474&keywords=software%20developer&f_TPR=r86400&f_TPR=r86400&origin=SEMANTIC_SEARCH_LANDING_PAGE&trackingId=n57Bi6chDK4BbpcL0ROwLA%3D%3D&refId=GthK0B8C8Dh0jgxHouOYXA%3D%3D&eBP=BUDGET_EXHAUSTED_JOB

2. Engineer
    Company: Target
    Location: N/A
    LinkedIn URL: https://www.linkedin.com/job

### Enhanced Job Scraping with Validation & Upload

In [ ]:
driver = webdriver.Chrome()
time.sleep(5)

try:
    actions.login(driver, email, password)
    print("✅ Login successful!")
except TimeoutException:
    print("⚠️ Login timeout - verification likely needed")
    print("Please complete any verification steps in the browser...")
    time.sleep(30)

print("🚀 Starting enhanced job scraping with validation & upload...")

try:
    # Job search configuration
    job_keywords = [
        "software developer",
        # "web developer", 
        # "python developer",
        # "frontend developer",
        # "backend developer",
        # "full stack developer",
        # "data scientist",
        # "machine learning engineer"
    ]
    
    all_jobs_data = []
    total_jobs_found = 0
    seen_job_urls = set()
    
    print(f"🔍 Searching for {len(job_keywords)} job types...")
    
    # Loop through each job keyword
    for job_idx, job_keyword in enumerate(job_keywords, 1):
        print(f"\n📋 [{job_idx}/{len(job_keywords)}] Searching: '{job_keyword}'")
        print("=" * 50)
        
        keyword_jobs_count = 0
        
        # Search first 3 pages for each keyword (reduced for demo)
        for page in range(3):
            start_param = page * 25
            encoded_keyword = job_keyword.replace(" ", "%20")
            job_search_url = f"https://www.linkedin.com/jobs/search-results/?f_TPR=r86400&keywords={encoded_keyword}&start={start_param}"
            
            print(f"  📄 Page {page + 1}/3")
            
            driver.get(job_search_url)
            time.sleep(8)
            
            # Scroll to load jobs
            for scroll in range(2):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
            
            # Extract job cards
            try:
                job_cards = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-job-id]"))
                )
                
                page_jobs = []
                
                for job_card in job_cards:
                    try:
                        # Extract job title and URL
                        job_title = "N/A"
                        job_url = "N/A"
                        
                        # Title selectors
                        title_selectors = [
                            "h3 a span[title]",
                            "h3 a",
                            ".base-search-card__title a",
                            "a[data-tracking-control-name*='job']"
                        ]
                        
                        for selector in title_selectors:
                            try:
                                elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if elem:
                                    job_title = elem.text.strip() or elem.get_attribute("title") or "N/A"
                                    if elem.tag_name == "a":
                                        job_url = elem.get_attribute("href") or "N/A"
                                    else:
                                        parent_link = elem.find_element(By.XPATH, "./ancestor::a[1]")
                                        job_url = parent_link.get_attribute("href") if parent_link else "N/A"
                                    if job_title != "N/A":
                                        break
                            except:
                                continue
                        
                        # Extract company name
                        company_name = "N/A"
                        company_selectors = [
                            ".base-search-card__subtitle a",
                            ".job-search-card__subtitle-link",
                            ".base-search-card__subtitle",
                            "h4 a",
                            "a[href*='/company/']"
                        ]
                        
                        for selector in company_selectors:
                            try:
                                elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if elem:
                                    company_text = (elem.text.strip() or 
                                                  elem.get_attribute("aria-label") or 
                                                  elem.get_attribute("title") or "").strip()
                                    
                                    if company_text and company_text not in ["N/A", "", "Company"]:
                                        company_name = company_text.replace("Company:", "").strip()
                                        break
                            except:
                                continue
                        
                        # Extract location
                        location = None
                        location_selectors = [
                            ".job-search-card__location",
                            ".base-search-card__metadata",
                        ]
                        
                        for selector in location_selectors:
                            try:
                                elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if elem:
                                    loc_text = elem.text.strip()
                                    if loc_text and loc_text != "N/A":
                                        location = loc_text
                                        break
                            except:
                                continue
                        
                        # Clean and validate URL
                        if job_url != "N/A" and job_url not in seen_job_urls:
                            # Clean LinkedIn URL (remove tracking parameters for cleaner storage)
                            if "linkedin.com/jobs" in job_url:
                                # Extract just the job ID part for cleaner URLs
                                import re
                                job_id_match = re.search(r'currentJobId=(\d+)', job_url)
                                if job_id_match:
                                    job_id = job_id_match.group(1)
                                    clean_url = f"https://www.linkedin.com/jobs/view/{job_id}/"
                                else:
                                    clean_url = job_url.split('&')[0] if '&' in job_url else job_url
                            else:
                                clean_url = job_url
                            
                            seen_job_urls.add(clean_url)
                            
                            job_data = {
                                'title': job_title,
                                'company': company_name,
                                'location': location,
                                'job_type': job_keyword,
                                'url': clean_url
                            }
                            page_jobs.append(job_data)
                        
                    except Exception:
                        continue  # Skip problematic jobs
                
                all_jobs_data.extend(page_jobs)
                keyword_jobs_count += len(page_jobs)
                
            except Exception as page_error:
                print(f"    ❌ Page {page + 1} error: {str(page_error)[:50]}...")
                continue
            
            time.sleep(2)  # Delay between pages
        
        print(f"  ✅ Found {keyword_jobs_count} unique jobs for '{job_keyword}'")
        total_jobs_found += keyword_jobs_count
        time.sleep(3)  # Delay between job types
    
    # Display summary
    print(f"\n🎯 SCRAPING COMPLETE!")
    print("=" * 60)
    print(f"📊 Total jobs scraped: {total_jobs_found}")
    print(f"🔄 Unique URLs collected: {len(seen_job_urls)}")
    
    # Show sample of scraped data
    if all_jobs_data:
        print(f"\n📝 Sample scraped jobs:")
        for i, job in enumerate(all_jobs_data[:5], 1):
            print(f"  {i}. {job['title']} at {job['company']}")
            if job['location']:
                print(f"     📍 {job['location']}")
            print(f"     🔗 {job['url'][:50]}...")
        
        if len(all_jobs_data) > 5:
            print(f"  ... and {len(all_jobs_data) - 5} more jobs")
    
    # Process and upload jobs
    print(f"\n🔄 Starting validation and upload process...")
    success = process_and_upload_jobs(all_jobs_data)
    
    if success:
        print(f"🎉 Job processing completed successfully!")
    else:
        print(f"⚠️ Job processing completed with issues. Check logs above.")
        
except TimeoutException:
    print("❌ Timeout error: LinkedIn anti-scraping measures detected")
    
except Exception as e:
    print(f"❌ Scraping error: {e}")
    
finally:
    try:
        print(f"\n⏸️ Press Enter to close browser...")
        input()
        driver.quit()
        print("✅ Browser closed successfully")
    except:
        print("⚠️ Browser was already closed")

### Test Validation & Sheet Integration

In [ ]:
# Test the validation and sheet integration with sample data
print("🧪 Testing JobModel validation and sheet integration...")

# Sample job data for testing (mix of valid and invalid)
test_jobs_data = [
    {
        'title': 'Senior Python Developer',
        'company': 'Google',
        'location': 'Mountain View, CA',
        'job_type': 'software developer',
        'url': 'https://www.linkedin.com/jobs/view/1234567/'
    },
    {
        'title': 'Frontend Engineer',
        'company': 'Facebook',
        'location': None,
        'job_type': 'web developer',
        'url': 'https://www.linkedin.com/jobs/view/7654321/'
    },
    {
        'title': 'N/A',  # Invalid title
        'company': 'Invalid Company',
        'location': 'New York, NY',
        'job_type': 'software developer',
        'url': 'https://www.linkedin.com/jobs/view/1111111/'
    },
    {
        'title': 'Data Scientist',
        'company': 'Microsoft',
        'location': 'Seattle, WA',
        'job_type': 'data science',
        'url': 'https://example.com/job123'  # Invalid URL (not LinkedIn)
    }
]

print(f"📋 Testing with {len(test_jobs_data)} sample jobs...")

try:
    # Get jobs sheet
    jobs_sheet = get_sheet_by_number(4)
    
    # Check current sheet structure
    columns = jobs_sheet.row_values(1)
    print(f"📊 Current sheet columns: {columns}")
    
    # Get existing data
    existing_urls = get_existing_job_urls(jobs_sheet)
    next_job_id = get_next_job_id(jobs_sheet)
    
    print(f"🔢 Next job ID: {next_job_id}")
    print(f"🔗 Existing URLs in sheet: {len(existing_urls)}")
    
    # Test validation
    validated_jobs = validate_and_process_jobs(test_jobs_data, existing_urls, next_job_id)
    
    print(f"\n✅ Validation test complete!")
    print(f"📊 Results: {len(validated_jobs)} valid jobs from {len(test_jobs_data)} test jobs")
    
    # Display validated jobs
    if validated_jobs:
        print(f"\n📝 Validated jobs:")
        for job in validated_jobs:
            print(f"  • ID {job.job_id}: {job.title} at {job.company}")
            print(f"    Location: {job.location or 'Not specified'}")
            print(f"    URL: {job.url}")
        
        # Ask user if they want to upload test data
        print(f"\n❓ Upload these test jobs to Google Sheets? (y/n): ", end="")
        upload_choice = input().lower().strip()
        
        if upload_choice == 'y':
            success = upload_jobs_to_sheet(jobs_sheet, validated_jobs)
            if success:
                print("✅ Test jobs uploaded successfully!")
            else:
                print("❌ Failed to upload test jobs")
        else:
            print("⏭️ Skipped uploading test jobs")
    
except Exception as e:
    print(f"❌ Test failed: {e}")
    import traceback
    traceback.print_exc()

### Simple Job Scraper - Fetch, Validate, Upload

In [ ]:
driver = webdriver.Chrome()
time.sleep(5)

try:
    # Login
    actions.login(driver, email, password)
    print("✅ Logged in successfully!")
    
    # Search jobs
    print("🔍 Searching jobs...")
    all_jobs = []
    
    job_search_url = "https://www.linkedin.com/jobs/search-results/?f_TPR=r86400&keywords=software%20developer"
    driver.get(job_search_url)
    time.sleep(8)
    
    # Scroll to load jobs
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Get job cards
    job_cards = driver.find_elements(By.CSS_SELECTOR, "[data-job-id]")
    print(f"📋 Found {len(job_cards)} job cards")
    
    # Extract job data
    for card in job_cards:
        try:
            # Get title and URL
            title_elem = card.find_element(By.CSS_SELECTOR, "h3 a")
            title = title_elem.text.strip()
            url = title_elem.get_attribute("href")
            
            # Get company
            company_elem = card.find_element(By.CSS_SELECTOR, ".base-search-card__subtitle a")
            company = company_elem.text.strip()
            
            # Get location (optional)
            try:
                location_elem = card.find_element(By.CSS_SELECTOR, ".job-search-card__location")
                location = location_elem.text.strip()
            except:
                location = None
            
            # Clean URL
            if "currentJobId=" in url:
                import re
                job_id = re.search(r'currentJobId=(\d+)', url).group(1)
                clean_url = f"https://www.linkedin.com/jobs/view/{job_id}/"
            else:
                clean_url = url
            
            job_data = {
                'title': title,
                'company': company,
                'location': location,
                'job_type': 'software developer',
                'url': clean_url
            }
            all_jobs.append(job_data)
            
        except Exception as e:
            continue
    
    print(f"✅ Extracted {len(all_jobs)} jobs")
    
    # Show sample
    if all_jobs:
        print("\n📝 Sample jobs:")
        for i, job in enumerate(all_jobs[:3]):
            print(f"  {i+1}. {job['title']} at {job['company']}")
    
    # Get sheet and validate
    jobs_sheet = get_sheet_by_number(4)
    existing_urls = get_existing_urls(jobs_sheet)
    
    print(f"\n📊 Sheet status: {len(existing_urls)} existing jobs")
    
    # Validate jobs
    validated_jobs = validate_jobs(all_jobs, existing_urls)
    print(f"✅ Validated: {len(validated_jobs)} new jobs")
    
    # Upload to sheet
    if validated_jobs:
        success = upload_to_sheet(jobs_sheet, validated_jobs)
        if success:
            print("🎉 Jobs uploaded successfully!")
        else:
            print("❌ Upload failed")
    else:
        print("⚠️ No new jobs to upload")
        
except Exception as e:
    print(f"❌ Error: {e}")
    
finally:
    try:
        input("Press Enter to close browser...")
        driver.quit()
        print("✅ Browser closed")
    except:
        pass

### Person data

In [4]:
driver = webdriver.Chrome()
time.sleep(5)
try:
    actions.login(driver, email, password)
    print("Login successful!")
except TimeoutException:
    print("Login timeout - verification likely needed")
    print("Please complete any verification steps in the browser...")
    time.sleep(60)

print("Proceeding with profile scraping...")
try:
    # Try scraping with error handling
    person = Person("https://www.linkedin.com/in/abhay-singh-patel-04b30b209/", 
                   driver=driver, scrape=True)
    print("✓ Profile scraped successfully!")
    print("="*80)
    print("COMPLETE CANDIDATE PROFILE")
    print("="*80)
    
    # Basic Information
    print(f"\n🔹 NAME: {person.name}")
    print(f"🔹 PROFILE URL: {person.linkedin_url}")
    print(f"🔹 CURRENT JOB TITLE: {person.job_title}")
    print(f"🔹 CURRENT COMPANY: {person.company}")
    
    # About Section
    print(f"\n📝 ABOUT:")
    print("-" * 40)
    print(f"{person.about}")
    
    # Work Experience
    print(f"\n💼 WORK EXPERIENCE:")
    print("-" * 40)
    if person.experiences:
        for i, exp in enumerate(person.experiences, 1):
            print(f"{i}. {exp.position_title} at {exp.institution_name}")
            if hasattr(exp, 'from_date') and exp.from_date:
                print(f"   📅 Duration: {exp.from_date} - {exp.to_date if exp.to_date else 'Present'}")
            if hasattr(exp, 'location') and exp.location:
                print(f"   📍 Location: {exp.location}")
            if hasattr(exp, 'description') and exp.description:
                print(f"   📋 Description: {exp.description}")
            print()
    else:
        print("No work experience information available")
    
    # Education
    print(f"\n🎓 EDUCATION:")
    print("-" * 40)
    if person.educations:
        for i, edu in enumerate(person.educations, 1):
            print(f"{i}. {edu.degree} at {edu.institution_name}")
            if hasattr(edu, 'from_date') and edu.from_date:
                print(f"   📅 Duration: {edu.from_date} - {edu.to_date if edu.to_date else 'Present'}")
            if hasattr(edu, 'description') and edu.description:
                print(f"   📋 Description: {edu.description}")
            print()
    else:
        print("No education information available")
    
    # Interests
    print(f"\n🎯 INTERESTS:")
    print("-" * 40)
    if person.interests:
        for i, interest in enumerate(person.interests, 1):
            print(f"{i}. {interest.title}")
    else:
        print("No interests information available")
    
    # Accomplishments
    print(f"\n🏆 ACCOMPLISHMENTS:")
    print("-" * 40)
    if person.accomplishments:
        for i, acc in enumerate(person.accomplishments, 1):
            print(f"{i}. {acc.title}")
            if hasattr(acc, 'description') and acc.description:
                print(f"   📋 {acc.description}")
    else:
        print("No accomplishments information available")
    
    # Additional Debug Information
    print(f"\n🔍 DEBUG INFO:")
    print("-" * 40)
    print(f"Total Experiences: {len(person.experiences) if person.experiences else 0}")
    print(f"Total Education entries: {len(person.educations) if person.educations else 0}")
    print(f"Total Interests: {len(person.interests) if person.interests else 0}")
    print(f"Total Accomplishments: {len(person.accomplishments) if person.accomplishments else 0}")
    
except Exception as e:
    print(f"✗ Error during scraping: {e}")
    print("This might be due to LinkedIn's anti-scraping measures.")
    print("Try the manual login approach or check if the profile is accessible.")
    
finally:
    driver.quit()

Login timeout - verification likely needed
Please complete any verification steps in the browser...
Proceeding with profile scraping...
✗ Error during scraping: name 'Person' is not defined
This might be due to LinkedIn's anti-scraping measures.
Try the manual login approach or check if the profile is accessible.


### Company data

In [ ]:
driver = webdriver.Chrome()
time.sleep(5)
try:
    actions.login(driver, email, password)
except TimeoutException:
    print("Login timeout - verification likely needed")
    print("Please complete any verification steps in the browser...")
    time.sleep(60)  # 1 minute to handle verification

print("Proceeding with Google company scraping...")
try:
    # Disable employee scraping to avoid timeout
    company = Company("https://www.linkedin.com/company/google/", driver=driver, get_employees=False)
    
    print("Company Information:")
    print(f"Name: {company.name}")
    print(f"About: {company.about_us}")
    print(f"Website: {company.website}")
    print(f"Headquarters: {company.headquarters}")
    print(f"Founded: {company.founded}")
    print(f"Company Type: {company.company_type}")
    print(f"Company Size: {company.company_size}")
    print(f"Specialties: {company.specialties}")
    
except Exception as e:
    print(f"Error scraping company: {e}")
    print("This might be due to LinkedIn's anti-scraping measures or network issues.")
    
finally:
    driver.quit()

### JOB data

In [ ]:
from linkedin_scraper import JobSearch, actions
from selenium import webdriver
import time

driver = webdriver.Chrome()

# Wait for page to load before attempting login
time.sleep(5)  

email = "santlalc27@gmail.com"
password = "Sant@123"

try:
    actions.login(driver, email, password)
except TimeoutException:
    print("Login timeout - verification likely needed")
    print("Please complete any verification steps in the browser...")
    time.sleep(30)  # 30 seconds to handle verification

print("Proceeding with job search scraping...")

try:
    # Define multiple job keywords to search
    job_keywords = [
        "software developer",
        # "web developer", 
        # "python developer",
        # "frontend developer",
        # "backend developer",
        # "full stack developer",
        # "data scientist",
        # "machine learning engineer"
    ]
    
    all_jobs_data = []
    total_jobs_found = 0
    seen_job_urls = set()  # To avoid duplicates
    
    print(f"🚀 Starting multi-job search for {len(job_keywords)} different job types...")
    
    # Loop through each job keyword
    for job_idx, job_keyword in enumerate(job_keywords, 1):
        print(f"\n🔍 [{job_idx}/{len(job_keywords)}] Searching for: '{job_keyword}'")
        print("=" * 60)
        
        keyword_jobs_count = 0
        
        # Loop through first 5 pages for each job keyword
        for page in range(5):
            start_param = page * 25
            # URL encode the job keyword
            encoded_keyword = job_keyword.replace(" ", "%20")
            job_search_url = f"https://www.linkedin.com/jobs/search-results/?f_TPR=r86400&keywords={encoded_keyword}&start={start_param}"
            
            print(f"  📄 Page {page + 1}/5 - {job_keyword}")
            
            driver.get(job_search_url)
            time.sleep(8)  # Wait for page to load
            
            # Scroll to load more jobs on the page
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_attempts = 0
            max_scroll_attempts = 2  # Reduced for efficiency
            
            while scroll_attempts < max_scroll_attempts:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    scroll_attempts += 1
                else:
                    scroll_attempts = 0  # Reset if we found new content
                last_height = new_height
            
            # Get job cards after scrolling
            try:
                job_cards = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-job-id]"))
                )
                
                page_jobs = []
                
                for i, job_card in enumerate(job_cards, 1):
                    try:
                        # Try multiple possible selectors for job title
                        job_title = "N/A"
                        job_url = "N/A"
                        
                        # Try various selectors for job title and link
                        title_selectors = [
                            "h3 a span[title]",
                            "h3 a",
                            ".base-search-card__title a",
                            ".job-card-container__link",
                            "a[data-tracking-control-name*='job']",
                            ".base-card__full-link",
                            "a span[aria-hidden='true']"
                        ]
                        
                        for selector in title_selectors:
                            try:
                                job_title_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if job_title_elem:
                                    job_title = job_title_elem.text.strip() or job_title_elem.get_attribute("title") or "N/A"
                                    if job_title_elem.tag_name == "a":
                                        job_url = job_title_elem.get_attribute("href") or "N/A"
                                    else:
                                        # Find parent link
                                        parent_link = job_title_elem.find_element(By.XPATH, "./ancestor::a[1]")
                                        job_url = parent_link.get_attribute("href") if parent_link else "N/A"
                                    if job_title != "N/A":
                                        break
                            except:
                                continue
                        
                        # Try to find company name with comprehensive selectors
                        company_name = "N/A"
                        company_selectors = [
                            # Primary company selectors
                            ".base-search-card__subtitle a",
                            ".job-search-card__subtitle-link",
                            ".job-card-container__company-name",
                            "h4 a",
                            
                            # Alternative company selectors
                            ".base-search-card__subtitle",
                            ".job-search-card__subtitle",
                            "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            "span[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            
                            # Generic selectors
                            "[data-tracking-control-name*='company']",
                            ".job-card-container__primary-description",
                            "a[href*='/company/']",
                            
                            # Fallback selectors
                            ".artdeco-entity-lockup__subtitle",
                            ".job-result-card__subtitle",
                            ".jobs-search-results-list__subtitle"
                        ]
                        
                        for selector in company_selectors:
                            try:
                                company_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if company_elem:
                                    # Try text first, then aria-label, then title attribute
                                    company_text = (company_elem.text.strip() or 
                                                  company_elem.get_attribute("aria-label") or 
                                                  company_elem.get_attribute("title") or "").strip()
                                    
                                    # Clean up common prefixes/suffixes
                                    if company_text and company_text not in ["N/A", "", "Company"]:
                                        # Remove common prefixes like "Company: "
                                        company_text = company_text.replace("Company:", "").strip()
                                        company_name = company_text
                                        break
                            except:
                                continue
                        
                        # If still no company found, try finding it in nearby elements
                        if company_name == "N/A":
                            try:
                                # Look for company info in sibling or parent elements
                                all_text_elements = job_card.find_elements(By.CSS_SELECTOR, "a, span, div")
                                for elem in all_text_elements[:10]:  # Check first 10 elements only
                                    elem_text = elem.text.strip()
                                    href = elem.get_attribute("href") or ""
                                    
                                    # If element links to a company page, it's likely the company name
                                    if "/company/" in href and elem_text and len(elem_text) < 100:
                                        company_name = elem_text
                                        break
                            except:
                                pass
                        
                        # Try to find location with multiple selectors
                        location = "N/A"
                        location_selectors = [
                            ".job-search-card__location",
                            ".base-search-card__metadata .job-card-container__metadata-item",
                            ".base-search-card__metadata",
                            "[data-tracking-control-name*='location']"
                        ]
                        
                        for selector in location_selectors:
                            try:
                                location_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if location_elem:
                                    location = location_elem.text.strip()
                                    if location:
                                        break
                            except:
                                continue
                        
                        # Check for duplicates using URL
                        if job_url != "N/A" and job_url not in seen_job_urls:
                            seen_job_urls.add(job_url)
                            
                            # Store job data
                            job_data = {
                                'title': job_title,
                                'company': company_name,
                                'location': location,
                                'url': job_url,
                                'job_type': job_keyword,
                                'page': page + 1
                            }
                            page_jobs.append(job_data)
                        
                    except Exception as job_error:
                        continue  # Skip problematic jobs
                
                all_jobs_data.extend(page_jobs)
                keyword_jobs_count += len(page_jobs)
                
            except Exception as page_error:
                print(f"    ❌ Error on page {page + 1}: {str(page_error)[:50]}...")
                continue
            
            # Small delay between pages
            time.sleep(2)
        
        print(f"  ✅ Found {keyword_jobs_count} unique jobs for '{job_keyword}'")
        total_jobs_found += keyword_jobs_count
        
        # Delay between different job searches
        time.sleep(3)
    
    # Display all collected jobs organized by job type
    print(f"\n🎯 FINAL RESULTS: {total_jobs_found} UNIQUE JOBS FROM {len(job_keywords)} JOB TYPES")
    print("=" * 100)
    
    # Group jobs by job type for better organization
    jobs_by_type = {}
    for job in all_jobs_data:
        job_type = job['job_type']
        if job_type not in jobs_by_type:
            jobs_by_type[job_type] = []
        jobs_by_type[job_type].append(job)
    
    # Display summary by job type
    print(f"\n📊 SUMMARY BY JOB TYPE:")
    for job_type, jobs in jobs_by_type.items():
        print(f"  • {job_type}: {len(jobs)} jobs")
    
    # Display all jobs
    job_counter = 1
    for job_type, jobs in jobs_by_type.items():
        print(f"\n\n🔹 {job_type.upper()} ({len(jobs)} jobs)")
        print("-" * 60)
        
        for job in jobs:
            print(f"{job_counter}. {job['title']}")
            print(f"    Company: {job['company']}")
            print(f"    Location: {job['location']}")
            print(f"    LinkedIn URL: {job['url']}" if len(job['url']) > 80 else f"    LinkedIn URL: {job['url']}")
            print()
            job_counter += 1
    
    print(f"🎉 Successfully scraped {total_jobs_found} unique job listings from {len(job_keywords)} job types!")
    print(f"📊 Jobs were collected from up to 5 pages per job type.")
    print(f"🚫 Duplicates removed: {len(seen_job_urls) - total_jobs_found} duplicate URLs filtered out.")
        
except TimeoutException as te:
    print(f"Timeout error: Could not load job search page")
    print("This might be due to LinkedIn's anti-scraping measures or slow page loading.")
    
except Exception as e:
    print(f"Error during job search: {e}")
    print("LinkedIn may have updated their page structure or implemented new anti-scraping measures.")
    
finally:
    # Always close the driver
    try:
        input("Press Enter to close the browser...")  # Keep browser open to see results
        driver.quit()
        print("Browser closed successfully.")
    except:
        print("Browser was already closed.")

### Jobs V2

In [ ]:

driver = webdriver.Chrome()

# Wait for page to load before attempting login
time.sleep(5)  

# email = "santlalc27@gmail.com"
# password = "Sant@123"

try:
    actions.login(driver, email, password)
except TimeoutException:
    print("Login timeout - verification likely needed")
    print("Please complete any verification steps in the browser...")
    time.sleep(30)  # 30 seconds to handle verification

print("Proceeding with job search scraping...")

try:
    # Define multiple job keywords to search
    job_keywords = [
        "software developer",
        # "web developer", 
        # "python developer",
        # "frontend developer",
        # "backend developer",
        # "full stack developer",
        # "data scientist",
        # "machine learning engineer"
    ]
    
    all_jobs_data = []
    total_jobs_found = 0
    seen_job_urls = set()  # To avoid duplicates
    
    print(f"🚀 Starting multi-job search for {len(job_keywords)} different job types...")
    
    # Loop through each job keyword
    for job_idx, job_keyword in enumerate(job_keywords, 1):
        print(f"\n🔍 [{job_idx}/{len(job_keywords)}] Searching for: '{job_keyword}'")
        print("=" * 60)
        
        keyword_jobs_count = 0
        
        # Loop through first 5 pages for each job keyword
        for page in range(5):
            start_param = page * 25
            # URL encode the job keyword
            encoded_keyword = job_keyword.replace(" ", "%20")
            job_search_url = f"https://www.linkedin.com/jobs/search-results/?f_TPR=r86400&keywords={encoded_keyword}&start={start_param}"
            
            print(f"  📄 Page {page + 1}/5 - {job_keyword}")
            
            driver.get(job_search_url)
            time.sleep(8)  # Wait for page to load
            
            # Scroll to load more jobs on the page
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_attempts = 0
            max_scroll_attempts = 2  # Reduced for efficiency
            
            while scroll_attempts < max_scroll_attempts:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    scroll_attempts += 1
                else:
                    scroll_attempts = 0  # Reset if we found new content
                last_height = new_height
            
            # Get job cards after scrolling
            try:
                job_cards = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-job-id]"))
                )
                
                page_jobs = []
                
                for i, job_card in enumerate(job_cards, 1):
                    try:
                        # Try multiple possible selectors for job title
                        job_title = "N/A"
                        job_url = "N/A"
                        
                        # Try various selectors for job title and link
                        title_selectors = [
                            "h3 a span[title]",
                            "h3 a",
                            ".base-search-card__title a",
                            ".job-card-container__link",
                            "a[data-tracking-control-name*='job']",
                            ".base-card__full-link",
                            "a span[aria-hidden='true']"
                        ]
                        
                        for selector in title_selectors:
                            try:
                                job_title_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if job_title_elem:
                                    job_title = job_title_elem.text.strip() or job_title_elem.get_attribute("title") or "N/A"
                                    if job_title_elem.tag_name == "a":
                                        job_url = job_title_elem.get_attribute("href") or "N/A"
                                    else:
                                        # Find parent link
                                        parent_link = job_title_elem.find_element(By.XPATH, "./ancestor::a[1]")
                                        job_url = parent_link.get_attribute("href") if parent_link else "N/A"
                                    if job_title != "N/A":
                                        break
                            except:
                                continue
                        
                        # Try to find company name with comprehensive selectors
                        company_name = "N/A"
                        company_selectors = [
                            # Primary company selectors
                            ".base-search-card__subtitle a",
                            ".job-search-card__subtitle-link",
                            ".job-card-container__company-name",
                            "h4 a",
                            
                            # Alternative company selectors
                            ".base-search-card__subtitle",
                            ".job-search-card__subtitle",
                            "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            "span[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']",
                            
                            # Generic selectors
                            "[data-tracking-control-name*='company']",
                            ".job-card-container__primary-description",
                            "a[href*='/company/']",
                            
                            # Fallback selectors
                            ".artdeco-entity-lockup__subtitle",
                            ".job-result-card__subtitle",
                            ".jobs-search-results-list__subtitle"
                        ]
                        
                        for selector in company_selectors:
                            try:
                                company_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if company_elem:
                                    # Try text first, then aria-label, then title attribute
                                    company_text = (company_elem.text.strip() or 
                                                  company_elem.get_attribute("aria-label") or 
                                                  company_elem.get_attribute("title") or "").strip()
                                    
                                    # Clean up common prefixes/suffixes
                                    if company_text and company_text not in ["N/A", "", "Company"]:
                                        # Remove common prefixes like "Company: "
                                        company_text = company_text.replace("Company:", "").strip()
                                        company_name = company_text
                                        break
                            except:
                                continue
                        
                        # If still no company found, try finding it in nearby elements
                        if company_name == "N/A":
                            try:
                                # Look for company info in sibling or parent elements
                                all_text_elements = job_card.find_elements(By.CSS_SELECTOR, "a, span, div")
                                for elem in all_text_elements[:10]:  # Check first 10 elements only
                                    elem_text = elem.text.strip()
                                    href = elem.get_attribute("href") or ""
                                    
                                    # If element links to a company page, it's likely the company name
                                    if "/company/" in href and elem_text and len(elem_text) < 100:
                                        company_name = elem_text
                                        break
                            except:
                                pass
                        
                        # Try to find location with multiple selectors
                        location = "N/A"
                        location_selectors = [
                            ".job-search-card__location",
                            ".base-search-card__metadata .job-card-container__metadata-item",
                            ".base-search-card__metadata",
                            "[data-tracking-control-name*='location']"
                        ]
                        
                        for selector in location_selectors:
                            try:
                                location_elem = job_card.find_element(By.CSS_SELECTOR, selector)
                                if location_elem:
                                    location = location_elem.text.strip()
                                    if location:
                                        break
                            except:
                                continue
                        
                        # Check for duplicates using URL
                        if job_url != "N/A" and job_url not in seen_job_urls:
                            seen_job_urls.add(job_url)
                            
                            # Store job data
                            job_data = {
                                'title': job_title,
                                'company': company_name,
                                'location': location,
                                'url': job_url,
                                'job_type': job_keyword,
                                'page': page + 1
                            }
                            page_jobs.append(job_data)
                        
                    except Exception as job_error:
                        continue  # Skip problematic jobs
                
                all_jobs_data.extend(page_jobs)
                keyword_jobs_count += len(page_jobs)
                
            except Exception as page_error:
                print(f"    ❌ Error on page {page + 1}: {str(page_error)[:50]}...")
                continue
            
            # Small delay between pages
            time.sleep(2)
        
        print(f"  ✅ Found {keyword_jobs_count} unique jobs for '{job_keyword}'")
        total_jobs_found += keyword_jobs_count
        
        # Delay between different job searches
        time.sleep(3)
    
    # Display all collected jobs organized by job type
    print(f"\n🎯 FINAL RESULTS: {total_jobs_found} UNIQUE JOBS FROM {len(job_keywords)} JOB TYPES")
    print("=" * 100)
    
    # Group jobs by job type for better organization
    jobs_by_type = {}
    for job in all_jobs_data:
        job_type = job['job_type']
        if job_type not in jobs_by_type:
            jobs_by_type[job_type] = []
        jobs_by_type[job_type].append(job)
    
    # Display summary by job type
    print(f"\n📊 SUMMARY BY JOB TYPE:")
    for job_type, jobs in jobs_by_type.items():
        print(f"  • {job_type}: {len(jobs)} jobs")
    
    # Display all jobs
    job_counter = 1
    for job_type, jobs in jobs_by_type.items():
        print(f"\n\n🔹 {job_type.upper()} ({len(jobs)} jobs)")
        print("-" * 60)
        
        for job in jobs:
            print(f"{job_counter}. {job['title']}")
            print(f"    Company: {job['company']}")
            print(f"    Location: {job['location']}")
            print(f"    LinkedIn URL: {job['url']}" if len(job['url']) > 80 else f"    LinkedIn URL: {job['url']}")
            print()
            job_counter += 1
    
    print(f"🎉 Successfully scraped {total_jobs_found} unique job listings from {len(job_keywords)} job types!")
    print(f"📊 Jobs were collected from up to 5 pages per job type.")
    print(f"🚫 Duplicates removed: {len(seen_job_urls) - total_jobs_found} duplicate URLs filtered out.")
        
except TimeoutException as te:
    print(f"Timeout error: Could not load job search page")
    print("This might be due to LinkedIn's anti-scraping measures or slow page loading.")
    
except Exception as e:
    print(f"Error during job search: {e}")
    print("LinkedIn may have updated their page structure or implemented new anti-scraping measures.")
    
finally:
    # Always close the driver
    try:
        input("Press Enter to close the browser...")  # Keep browser open to see results
        driver.quit()
        print("Browser closed successfully.")
    except:
        print("Browser was already closed.")

### Testing the New Jobs Package

In [ ]:
# Import the new jobs package
from jobs import main, get_jobs, JobModel, get_sheet_by_number

# Test the package - scrape jobs and process them
print("Testing the jobs package...")

# Scrape jobs using the package
print("1. Scraping jobs...")
scraped_jobs = get_jobs()

print(f"2. Scraped {len(scraped_jobs)} jobs")

# Process and upload using the main pipeline
print("3. Processing and uploading jobs...")
main(scraped_jobs)

print("Package test completed!")